In [30]:
import re

#1. 정규 표현식으로 IP 주소 형식 패턴 정의(범주X)
ip_pattern = r"(?:\d{1,3}\.){3}\d{1,3}"

# 'IP_LIKE' 열 추가
df['IP_LIKE'] = pd.Series(dtype=int)

# URL 문자열 내 IP 주소 형식 존재 여부 확인 및 'IP_LIKE' 열 업데이트
for idx, url in df['url'].items():
    if re.search(ip_pattern, url):
        df.loc[idx, 'IP_LIKE'] = 1
    else:
        df.loc[idx, 'IP_LIKE'] = 0

In [31]:
#2.Checks the presence of @ in URL (HaveAt)(범주X)
df.loc[:, 'AT'] = pd.Series(dtype=int)

def HaveAt(url):
    if "@" in url:
        at = 1
    else:
        at = 0
    return at

# Apply the function to each URL and update 'AT' column
df.loc[:, 'AT'] = df['url'].apply(HaveAt)

In [32]:
from urllib.parse import urlparse

#3. Gives number of '/' in URL (URL_Depth)(범주X)
df.loc[:, 'URL_Depth'] = pd.Series(dtype=int)

def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth
    
# Apply the function to each URL and update 'URL_Depth' column
df.loc[:, 'URL_Depth'] = df['url'].apply(getDepth)

In [33]:
#4.Checking for redirection '//' in the url (Redirection)

# 새로운 'Redirection' 열을 추가하고 int 형으로 초기화(범주 0)
df.loc[:, 'Redirection'] = pd.Series(dtype=int)

def redirection(url):
    pos = url.rfind('//')  # URL 내에서 마지막으로 나타나는 '//'의 위치를 찾기
    if pos > 6:  # 위치가 6보다 크다면
        if pos > 7:  # 위치가 7보다 크다면 (즉, 프로토콜 부분을 넘어서 존재하는 경우)
            return 1  # 리디렉션이 의심되는 URL로 간주하여 1 반환
        else:
            return 0  # 그렇지 않다면 0 반환(의심)
    else:
        return 0  # '//'가 프로토콜 부분에만 있는 경우 0 반환(의심)

# 각 URL에 대해 redirection 함수를 적용하고 'Redirection' 열을 업데이트
df.loc[:, 'Redirection'] = df['url'].apply(redirection)

In [43]:
#5.URL의 스킴이 'https'인지(범주X)
import urllib.parse

def is_https(url):
    return 1 if urllib.parse.urlsplit(url).scheme == 'https' else 0

df.loc[:, 'Is_Https'] = df['url'].apply(is_https)

Is_Https
1           45145
0            4855
Name: count, dtype: int64

In [49]:
import re

#6.1 20가지의 주요한 url 단축 서비스 패턴
shorteningServices = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net|buff\.ly|rb\.gy|rebrand\.ly|short\.cm|clk\.im|cutt\.ly|t2m\.io|bl\.ink|" \
                      r"tiny\.cc"    #단축서비스 패턴 10개 추가함

df.loc[:, 'TINY_URL'] = pd.Series(dtype=int)

#6.2 url에 단축서비스가 포함되어있는 지 확인(범주X)
def tinyURL(url):
    match=re.search(shorteningServices,url) #정규 표현식을 사용하여 입력된 url에서 단축서비스 패턴을 찾음
    if match: #입력된 url에 단축서비스 패턴이 있으면 1을 리턴, 아니면 0을 리턴
        return 1
    else:
        return 0

# Apply the function to each URL and update 'TINY_URL' column
df.loc[:, 'TINY_URL'] = df['url'].apply(tinyURL)

TINY_URL
0.0         47631
1.0          2369
Name: count, dtype: int64

In [ ]:
#7. -이 있는지 (범주X)
def check_Hyphen(domain):
    if '-' in domain:
        return 1 
    else:
        return 0 

data['Check_Hyphen'] = data['url'].apply(check_phishing)

In [ ]:
#8.쿼리문자열의 개수 (범주X)

def parse_query_string(url):
    # URL에서 '?' 문자열이 있는지 확인하여 쿼리 문자열이 있는지 판별
    if '?' not in url:
        return 0
    
    # URL에서 쿼리 문자열 추출
    query_string = url.split('?')[-1]
    
    # 쿼리 문자열을 '&'로 분할하여 각 쌍을 추출
    query_pairs = query_string.split('&')
    
    # 각 쌍을 이름과 값으로 분할하여 딕셔너리에 저장
    params = {}
    for pair in query_pairs:
        key, value = pair.split('=')
        params[key] = value
    
    return params


def Query(url):
    query_params = parse_query_string(url)
    if query_params is not None:
        return len(query_params)
    else:
        return 0

# 쿼리 문자열의 개수를 업데이트
df['Query'] = df['url'].apply(Query)

In [ ]:
#9.도메인 생성일 기준 (범주)
import whois
from datetime import datetime, timedelta
import socket

# 타임아웃 시간(초) 설정
TIMEOUT = 5

def is_domain_created(url):
    try:
        domain_name = urllib.parse.urlsplit(url).netloc
        socket.setdefaulttimeout(TIMEOUT)  # 타임아웃 설정
        domain_info = whois.whois(domain_name)
        creation_date = domain_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        today = datetime.today()
        one_years_ago = today - timedelta(days=365)
        two_years_ago = today - timedelta(dats=730)
        if creation_date and creation_date >= one_years_ago and creation_date <= two_years_ago :
            return 1 # 1년 이하(피싱 사이트)
        else:
            return 0  # 2년 이상(피싱 사이트 의심)
    except (whois.parser.PywhoisError, socket.timeout) as e:
        return 1  # 오류 발생 시 1(피싱 사이트라고 간주)


df.loc[:, 'Domain_Age'] = pd.Series(dtype=int)

# Apply the function to each URL and update 'Domain_Age' column
df.loc[:, 'Domain_Age'] = df['url'].apply(is_domain_created)

In [ ]:
#10.도메인 만료일 기준 (범주)
import whois
from datetime import datetime, timedelta
from urllib.parse import urlparse
import socket
# 타임아웃 시간(초) 설정
TIMEOUT = 5

# 링크에서 도메인을 추출하는 함수
def get_domain_from_link(link):
    # 링크를 구문 분석하여 도메인 부분을 추출
    parsed_uri = urlparse(link)
    domain = '{uri.netloc}'.format(uri=parsed_uri)
    return domain

# 도메인 만료 여부를 확인하는 함수
def domain_end(domain_name):
    try:
        # 타임아웃 설정
        socket.setdefaulttimeout(TIMEOUT)
        
        # 도메인 정보를 가져옴
        domain_info = whois.whois(domain_name)
        
        # 만료일 가져옴
        expiration_date = domain_info.expiration_date
        
        # 만료일이 문자열인 경우, 날짜 형식으로 변환
        if isinstance(expiration_date, str):
            expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")
        
        # 만료일이 없는 경우 또는 리스트 형태인 경우에는 0을 반환
        if expiration_date is None or isinstance(expiration_date, list):
            return 0  # 피싱 사이트 의심
        else:
            # 현재 날짜 가져오기
            today = datetime.now()
            # 만료일까지의 남은 일수를 계산
            days_until_expiry = (expiration_date - today).days
            # 남은 일수를 개월 수로 변환해 6개월 이내인지 확인
            if (days_until_expiry / 30) < 6:
                return 1  # 6개월 이내(피싱 사이트)
            else:
                return 0  # 6개월 이상(의심)
    except (whois.parser.PywhoisError, socket.timeout):
        # 오류가 발생한 경우 1을 반환(피싱 사이트라고 간주)
        return 1

# 각 URL에 대해 도메인 만료 여부를 확인하는 함수
def is_domain_created_within_six_months(url):
    domain_name = get_domain_from_link(url)
    return domain_end(domain_name)


# 'Domain_end' 컬럼 생성 및 함수 적용
df['Domain_end'] = df['url'].apply(is_domain_created_within_six_months)

In [ ]:
#11. Mouse_over (범주X)
import re
import requests
from bs4 import BeautifulSoup

def check_mouseover(html_content):
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    # onmouseover 속성을 가진 태그를 찾음
    if soup.find(attrs={"onmouseover": True}):
        return 1  # 마우스 오버 발생
    else:
        return 0  # 마우스 오버 발생 X

def check_url(url):
    try:
        response = requests.get(url, timeout=5)  # 타임아웃을 5초로 설정
        response.raise_for_status()  # HTTP 에러가 발생하면 예외 발생
        response_text = response.text
        result = check_mouseover(response_text)
    except requests.exceptions.RequestException as e:
        result = -1  # 타임아웃 또는 다른 요청 예외 발생 시 -1을 반환
    return result


# 'url' 컬럼의 각 URL에 대해 Mouseover 여부를 확인하여 'Mouseover' 컬럼에 결과를 저장
df['Mouseover'] = df['url'].apply(check_url)

In [ ]:
#12. Web_Forwards(범주)
import requests

def web_forwards(url, timeout=5):
    try:
        respon = requests.get(url, allow_redirects=True, timeout=timeout)
    except requests.RequestException:
        return 1  # 요청 중 오류가 발생하면 피싱 사이트로 간주

    if not respon.history:
        return 0  # 리디렉션이 전혀 없는 경우 의심 사이트 

    # 리디렉션 횟수가 2회 이하인 경우 의심 사이트로 간주
    if len(respon.history) <= 2:
        return 0
    else:
        return 1  # 리디렉션이 3회 이상인 경우 피싱 사이트로 간주

# 'url' 컬럼의 각 URL에 대해 리디렉션 여부를 확인하여 'Web_forwards' 컬럼에 결과를 저장
df['Web_forwards'] = df['url'].apply(web_forwards)

In [ ]:
#13.구글 검색 여부 (범주X)
import requests
import pandas as pd

def is_indexed_in_google(url):
    try:
        response = requests.get(f"https://www.google.com/search?q=site:{url}")
        if response.status_code == 200:
            if "did not match any documents" in response.text:
                return False
            else:
                return True
        else:
            #Failed to retrieve search results from Google
            return None
    except Exception as e:
        #error
        return None
        
# 구글에 인덱싱되었는지 여부를 나타내는 새로운 열 추가
df['Google_search'] = df['url'].apply(is_indexed_in_google)

# 1과 0으로 구글 검색 여부를 인코딩하여 업데이트
df['Google_search'] = df['Google_search'].apply(lambda x: 1 if x else 0)

In [ ]:
#14. URL이 실제로 갖고 있는 <a> 태그의 수 (범주X)
import requests
from bs4 import BeautifulSoup

# 타임아웃 시간(초) 설정
TIMEOUT = 5

def count_hyperlinks(url):
    try:
        response = requests.get(url, timeout=TIMEOUT)
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.content, 'html.parser')
        hyperlinks = soup.find_all('a')
        return len(hyperlinks)
    except (requests.RequestException, requests.Timeout):
        return -1  # 요청 실패 시 None 반환


# 하이퍼링크의 총 개수를 업데이트
df['Hyperlinks'] = df['url'].apply(count_hyperlinks)

In [ ]:
#15. 도메인 일관성을 확인하는 함수(범주) 
def check_domain_consistency(url):
    try:
        response = requests.get(url, timeout=5)
        original_domain = urlparse(url).netloc
        final_domain = urlparse(response.url).netloc
        is_same_domain = 1 if original_domain == final_domain else 0
        return is_same_domain
    except requests.RequestException as e:
        return 0

df['Domain_Cons'] = df['url'].apply(check_domain_consistency)

In [ ]:
#16. 최종 URL 길이를 반환하는 함수(범주X)
def get_final_url_length(url):
    try:
        response = requests.get(url, timeout=5)
        final_url_length = len(response.url)
        return final_url_length
    except requests.RequestException as e:
        return -1 #error
        
df['URL_length'] = df['url'].apply(get_final_url_length)

In [ ]:
#17. HTTP 상태 코드 리스트를 반환하는 함수(범주X)
def get_http_status_codes(url):
    try:
        response = requests.get(url, timeout=5)
        # 최종 응답의 상태 코드만 반환
        return response.status_code
        
    except requests.RequestException as e:
        return 0 #반환을 못하는 경우 

df['HTTP_Code'] = df['url'].apply(get_http_status_codes)

In [ ]:
#18. HTTP 상태 코드 리스트를 반환하는 함수(범주)
def get_http_status_codes(url):
    try:
        response = requests.get(url, timeout=5)
        # 최종 응답의 상태 코드만 반환
        return response.status_code
    except requests.RequestException as e:
        return None #error

# 주어진 URL에 대해 피처를 추출하는 함수
def phishing_likelihood(url):
    try:
        # HTTP 상태 코드를 가져옴
        http_status_codes = get_http_status_codes(url)
        if http_status_codes is None:
            return 1 #error(피싱 사이트라고 간주)
        
        # 피싱 사이트일 가능성을 나타내는 피처 초기화
        phishing_likelihood = 0
        
        # 피싱 사이트일 가능성을 판단하여 피처 설정
        if any(code >= 300 for code in http_status_codes):
            phishing_likelihood = 1
        else: 
            return 0 #의심 

    except Exception as e:
        return 1 #error

df['HTTP_Status'] = df['url'].apply(get_http_status_codes)